# 特徴量の生成と変換

## 特徴量生成
既存の特徴量から新しい意味のある特徴量を導き出して、モデルがデータ内の重要なパターンをより良く捉えられるようにする。

特徴量生成の例：
- 日時機能
- インタラクション機能（複数の特徴量を組み合わせる）
- 集計（顧客の購入頻度と平均購入額を掛け合わせる）

## 特徴量変換
既存の特徴量を変更して、学習アルゴリズムにより適合させる手法

**一般的な変換方法**
- 対数変換（Logarithmic Transformation）
    - 強く歪んだ分布における歪度を軽減

- 平方根変換（Square Root Transformation）
    - 中程度の歪度を軽減、特にカウントデータに使用されることが多い

- 多項式変換（Polynomial Transformation）
    - 高次項（$𝑥^2, 𝑥^3$ など）を追加して非線形関係を捉える

**重要性**
- モデルの非線形関係への適合能力を向上させる
- 分布をより正規分布に近づけ、正規性を仮定するアルゴリズムの性能向上に寄与

**非線形関係における特徴量変換の重要性**

* 特徴量変換により、線形モデルが非線形関係を扱えるようになる

  * 例：

    * 多項式変換は、線形回帰で二次的パターンをモデル化できるようにする
    * 対数変換は、分散を安定させ、歪度を処理する

* 特徴量を変換することで、モデルはより堅牢になり、データ内の複雑なパターンを捉える能力が向上する

---
## 演習
1. 日付列から新しい特徴量を作成する（例：曜日、月、年）
2. データセットに多項式変換を適用し、変換前後でモデル性能を比較する

In [30]:
import pandas as pd

In [31]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

### Load Data

In [32]:

df = pd.read_csv("bike_sharing_daily.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  atemp       731 non-null    float64
 11  hum         731 non-null    float64
 12  windspeed   731 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB


In [33]:
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


### 1. Create new features from a date column

In [34]:
# Convert dteday to datetime
df['dteday'] = pd.to_datetime(df['dteday'])

# Create new features
df['day_of_week'] = df['dteday'].dt.day_name()
df['month'] = df['dteday'].dt.month
df['year'] = df['dteday'].dt.year

# Display the new features
print("\nNew Features: ", df[['dteday', 'day_of_week', 'month', 'year']])


New Features:          dteday day_of_week  month  year
0   2011-01-01    Saturday      1  2011
1   2011-01-02      Sunday      1  2011
2   2011-01-03      Monday      1  2011
3   2011-01-04     Tuesday      1  2011
4   2011-01-05   Wednesday      1  2011
..         ...         ...    ...   ...
726 2012-12-27    Thursday     12  2012
727 2012-12-28      Friday     12  2012
728 2012-12-29    Saturday     12  2012
729 2012-12-30      Sunday     12  2012
730 2012-12-31      Monday     12  2012

[731 rows x 4 columns]


### 2a. Apply polynomial transformations to a dataset

In [ ]:
# Select feature and target
X = df[['temp']]
y = df['cnt']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
## Train Linear Regression model with the original dataset
model_original = LinearRegression()
model_original.fit(X_train, y_train)
mse_original = mean_squared_error(y_test, model_original.predict(X_test))

In [ ]:
# Apply polynomial transformation and train the model
poly = PolynomialFeatures(degree=2, include_bias=False)

X_poly_train = poly.fit_transform(X_train)  # fit は訓練データのみ
"""
poly.fit_transform(X) 
元の特徴量行列 X から、2次までのすべての項（単項・累乗・交互作用項）を計算して新しい特徴量集合を作る。
例：
[a, b] -> [a, b, a^2, ab, b^2].
[a, b, c] -> [a, b, c, a^2, ab, ac, b^2, bc, c^2]

目的：
モデルが非線形な関係や特徴の組み合わせ（交互作用）を学習できるようにするため、特徴空間を拡張する。
特に線形回帰モデルで多項式回帰（非線形カーブの近似）を行いたいときに使われる。

fit: 学習データ X_train を見て、多項式項を生成するルールを決定
transform: fitで決めたルールに基づいて、 X_train の値を変換して新しい行列(X_poly_train)を生成
-> X_poly_trainには 'temp', 'temp^2' の2列データが入る。
"""

X_poly_test  = poly.transform(X_test)       # テストは transform
"""
※学習データと同じ列構造（同じ特徴量数）の X_poly_test を得たいため、テストデータではfitは呼ばない。
（テストデータに対してfitを行うと、
　本来未知であるべきテストデータの情報が学習過程に混入してしまう＝「データリーク」）
"""

model_poly = LinearRegression()
model_poly.fit(X_poly_train, y_train)
mse_poly = mean_squared_error(y_test, model_poly.predict(X_poly_test))

### 2b. Compare model performance before and after transformation

In [38]:
# Compare results
print(f"MSE original: {mse_original:.2f}")
print(f"MSE Polynomial: {mse_poly:.2f}")

MSE original: 2391051.89
MSE Polynomial: 2431396.49
